# GLASO CORRELATION

# Solution Gas-Oil Ratio (Rs) Estimation Using the Glaso (1980) Correlation

The objective is to estimate the solution gas-oil ratio (R_s) using a modified version of the Glaso (1980) correlation. This method involves defining (R_s) as a function of reservoir parameters, such as gas gravity, API gravity, temperature, and pressure. The correlation relies on a set of optimization parameters `alpha_1` to `alpha_7` to minimize the difference between actual and estimated R_s values through optimization algorithms.


### Formula for Estimated R_s

The equation to estimate R_s is given by:

In [91]:
from sympy import*

# Defining the symbols
gamma_g, API, T, alpha_1, alpha_2, alpha_3, x = symbols('gamma_g API T alpha_1 alpha_2 alpha_3 x')

# Defining the equation for Rs
Rs = gamma_g * ((API ** alpha_1 / (T - 460) ** alpha_2) * 10 ** x) ** alpha_3

# Displaying the equation
Rs

gamma_g*(10**x*API**alpha_1/(T - 460)**alpha_2)**alpha_3

### Variables

- **\( R_s \)**: The solution gas-oil ratio. It represents the amount of gas dissolved in oil under reservoir conditions, measured in standard cubic feet per stock tank barrel (scf/STB).

- **\( \gamma_g \)**: The specific gravity of the gas. This is a dimensionless value representing the density of the gas relative to air, where air has a specific gravity of 1. A higher \( \gamma_g \) means the gas is denser.

- **API**: The API gravity of the oil, measured in degrees. API gravity is an industry-standard measurement to indicate how heavy or light a petroleum liquid is relative to water. The higher the API, the lighter the oil.

- **\( T \)**: The reservoir temperature in degrees Fahrenheit (°F). This temperature affects the solubility of gas in oil; generally, higher temperatures result in lower gas solubility.

- **\( \alpha_1, \alpha_2, \alpha_3 \)**: These are optimization parameters to be determined. Each of these parameters affects the relationship between API gravity, temperature, and solution gas-oil ratio:
    - **\( \alpha_1 \)**: Exponent for API gravity.
    - **\( \alpha_2 \)**: Exponent for temperature.
    - **\( \alpha_3 \)**: Exponent applied to the entire term within brackets to adjust the scaling.

- **\( x \)**: An intermediate variable calculated as part of the model, which depends on reservoir pressure and other factors. It is often computed separately using its own equation in the model.


In [103]:
import sympy as sp

# Defining symbols
alpha_4, alpha_5, alpha_6, alpha_7, P = sp.symbols('alpha_4 alpha_5 alpha_6 alpha_7 P')

# Defining the equation
x = alpha_4 - (alpha_5 - alpha_6 * sp.log(P)) ** alpha_7

# Displaying the equation
print("The formula for finding x is : ")
x

The formula for finding x is : 


alpha_4 - (alpha_5 - alpha_6*log(P))**alpha_7

where:
- P: Reservoir pressure in psia
- log: Logarithm base 10
- alpha_4, alpha_5, alpha_6, alpha_7: Optimization parameters to be determined


### Purpose of Each Variable

The equation aims to estimate the solution gas-oil ratio R_s by combining properties of the gas, oil, and reservoir conditions. The values of `alpha_1`, `alpha_2`, and `alpha_3` are optimized to fit experimental data and improve the accuracy of the model.

In [2]:
import numpy as np
from scipy.optimize import minimize

# Function to check if temperature and pressure are valid
def check_inputs(T, P):
    if T < 460:
        raise ValueError("Temperature must be greater than or equal to 460°F for valid calculations.")
    if P <= 0:
        raise ValueError("Pressure must be a positive value greater than zero for valid calculations.")

# User input with explicit instructions and examples
print("Please provide the following inputs. If you're unsure about any of the values, consult the instructions.")

gamma_g = float(input("Enter gas gravity (gamma_g):\n- Gas gravity is a measure of the density of gas relative to air.\n- Example: 0.743\n- Please enter a numeric value: "))
API = float(input("Enter API gravity (API):\n- API gravity is a measure of how heavy or light a petroleum liquid is compared to water.\n- Example: 37\n- Please enter a numeric value: "))
T = float(input("Enter temperature in Fahrenheit (T):\n- Temperature is required in Fahrenheit.\n- It should be greater than or equal to 460°F.\n- Example: 500\n- Please enter a numeric value: "))
P = float(input("Enter reservoir pressure in psia (P):\n- Pressure is required in psia (pounds per square inch absolute).\n- Example: 3000\n- Please enter a numeric value: "))

# Check temperature and pressure validity
try:
    check_inputs(T, P)
except ValueError as e:
    print(e)
    exit()  # Exit the program if inputs are invalid

# Actual Rs values from lab data (for demonstration)
Rs_actual_input = input("Enter actual Rs values (comma-separated):\n- Rs values represent the gas-to-oil ratio at the given conditions.\n- Example: 500,600,578\n- Please enter your actual values separated by commas: ")
Rs_actual = list(map(float, Rs_actual_input.split(',')))

# Define the model's estimated Rs function based on the formula
def calculate_rs_estimated(alpha, gamma_g, API, T, P):
    # Unpack the optimization parameters
    alpha_1, alpha_2, alpha_3, alpha_4, alpha_5, alpha_6, alpha_7 = alpha

    # Calculate x according to Equation 2 with checks for valid values
    try:
        x = alpha_4 - (alpha_5 - alpha_6 * np.log10(P)) ** alpha_7
    except ValueError:
        print("Invalid operation in x calculation")
        return np.nan

    # Calculate Rs (estimated) according to Equation 1 with checks
    try:
        Rs_estimated = gamma_g * ((API ** alpha_1 / (T - 460) ** alpha_2) * 10 ** x) ** alpha_3
    except ValueError:
        print("Invalid operation in Rs_estimated calculation")
        return np.nan
    
    return Rs_estimated

# Define the objective function for optimization
def objective_function(alpha, gamma_g, API, T, P, Rs_actual):
    errors = []
    for actual_rs in Rs_actual:
        estimated_rs = calculate_rs_estimated(alpha, gamma_g, API, T, P)
        
        # Check if Rs_estimated is valid
        if np.isnan(estimated_rs):
            return np.inf  # Return a high error if there was an invalid calculation

        error = abs(actual_rs - estimated_rs)
        errors.append(error)
    
    # Sum of absolute errors
    return sum(errors)

# Initial guess for the optimization parameters
initial_alpha = [1.0, 1.0, 1.0, 2.8869, 14.1811, 3.3053, 0.5]  # Initial guesses for alpha values

# Performing the optimization
result = minimize(objective_function, initial_alpha, args=(gamma_g, API, T, P, Rs_actual), method='L-BFGS-B')

# Optimized parameters are
optimized_alpha = result.x
print("Optimized parameters:", optimized_alpha)
print("Objective function value:", result.fun)


Please provide the following inputs. If you're unsure about any of the values, consult the instructions.


Enter gas gravity (gamma_g):
- Gas gravity is a measure of the density of gas relative to air.
- Example: 0.743
- Please enter a numeric value:  0.78
Enter API gravity (API):
- API gravity is a measure of how heavy or light a petroleum liquid is compared to water.
- Example: 37
- Please enter a numeric value:  88
Enter temperature in Fahrenheit (T):
- Temperature is required in Fahrenheit.
- It should be greater than or equal to 460°F.
- Example: 500
- Please enter a numeric value:  987
Enter reservoir pressure in psia (P):
- Pressure is required in psia (pounds per square inch absolute).
- Example: 3000
- Please enter a numeric value:  8900
Enter actual Rs values (comma-separated):
- Rs values represent the gas-to-oil ratio at the given conditions.
- Example: 500,600,578
- Please enter your actual values separated by commas:  899,980,898


Optimized parameters: [ 1.      1.      1.      2.8869 14.1811  3.3053  0.5   ]
Objective function value: 2750.872370574025


C:\Users\hp\AppData\Local\Temp\ipykernel_14820\3591688177.py:37: RuntimeWarning: invalid value encountered in scalar power
  x = alpha_4 - (alpha_5 - alpha_6 * np.log10(P)) ** alpha_7
C:\Users\hp\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
